In [1]:
# setup
# !mkdir krd_parks
# !conda create -n krd_parks python=3.9 
# !cd krd_parks
# !pip install openrouteservice folium

In [2]:
import folium
import json, requests
from openrouteservice import client

In [3]:
start_points = {'first': {'location': {'lat': 45.0737815284364, 'long': 39.02195956971909}},
                'second': {'location': {'lat': 45.0507, 'long': 39.0008}},
                'third': {'location': {'lat': 45.06489768105405, 'long': 38.93822527162066}},
                'fourth': {'location': {'lat': 45.00831632571033, 'long': 39.026149989325965}},
                'fifth': {'location': {'lat': 45.11889454019721, 'long': 38.97430825348612}}
}
relation_id = 269701 # Krasnodar relation id from OSM
url = f'http://polygons.openstreetmap.fr/get_geojson.py?id={relation_id}&params=0'
city_border_geojson = requests.get(url)

In [4]:
api_key = '5b3ce3597851110001cf6248588fc1fe85794b32933411db07d36f78'
clnt = client.Client(key=api_key)
map_kr = folium.Map(tiles='OpenStreetMap', location=([45.057761, 38.989971]), zoom_start=10)
s_points_group = folium.FeatureGroup(name='Start Points', show=False) # feature group to create layers later
for name, point in start_points.items():
    folium.map.Marker([point['location']['lat'], point['location']['long']],
                        icon=folium.Icon(color='red',
                                        icon_color='#34dbeb',
                                        icon='info-sign'
                                      ),
                        popup=name
                     ).add_to(s_points_group) # creating markers and adding them to feature group
s_points_group.add_to(map_kr) # adding group to map

In [5]:
border_group = folium.FeatureGroup(name="Border") # border layer (for final map)
folium.features.GeoJson(city_border_geojson.text).add_to(border_group)
border_group.add_to(map_kr)
pop = folium.LatLngPopup() 
map_kr.add_child(pop) # to add popup on click to show coordinates
map_kr

In [6]:
categories_poi = { # dict, if we will need more categories to search for
    'parks': [280]
}
params_poi = { # common request params
    'request': 'pois',
    'buffer': 1000
}
sp_geometry = dict() # dict to create geometry for points
sp_geometry['type'] = 'Point'
print(f"Points buffer is: {params_poi['buffer']} meters.")
for name, point in start_points.items():
    point['pois'] = dict() # dict to store pois
    sp_geometry['coordinates'] = [point['location']['long'], point['location']['lat']] # folium and ORS have reverse 
                                                                                    # coordinates order
    params_poi['geojson'] = point['geometry'] = sp_geometry # geometry for request and stroting it to point
    print(f'POIs around the {name} point:')
    
    for cat_name, category in categories_poi.items():
        params_poi['filter_category_ids'] = category # categories to request for 
        point['pois'][cat_name] = dict() # dict to store pois data depending on poi category
        point['pois'][cat_name]['geojson'] = clnt.places(**params_poi)[0]['features'] # request to ORS API
        print(f"\t{cat_name} -> {len(point['pois'][cat_name]['geojson'])}")

Points buffer is: 1000 meters.
POIs around the first point:
	parks -> 5
POIs around the second point:
	parks -> 7
POIs around the third point:
	parks -> 2
POIs around the fourth point:
	parks -> 0
POIs around the fifth point:
	parks -> 0


In [8]:
# with open('geo-magnit.geojson', 'w') as fp:
#     json.dump(start_points, fp)

In [9]:
cat_f_groups = dict() # dict for folium.FeatureGrops for every category (to add layer on map)
for name, point in start_points.items():
    for cat_name in categories_poi.keys():
        if cat_f_groups.get(cat_name) == None:
            cat_f_groups[cat_name] = folium.FeatureGroup(name=cat_name)
        for poi in point['pois'][cat_name]['geojson']:
            folium.map.Marker(list(reversed(poi['geometry']['coordinates'])),
                                    icon=folium.Icon(color='green',
                                         icon_color='#ff9e36',
                                         icon='tree',
                                         prefix='fa'
                                    ),
                                    popup=f"distance from {name}: {poi['properties']['distance']}"
            ).add_to(cat_f_groups[cat_name])
            
for f_group in cat_f_groups.values():
    f_group.add_to(map_kr) # adding feature groups to map
    
folium.LayerControl().add_to(map_kr) # adding layer control to map
map_kr # map has layer control in top-right corner